In [1]:
import pandas   as pd

/tmp/ipykernel_19735/3653822576.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas   as pd


In [17]:
df = pd.read_csv('./bitcoin.csv', index_col=False)
print(len(df))
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
df = df[['rank', 'exchangeId', 'exchangeName', 'exchangeSlug', 'volumePercent', 'volumeQuote', 'isVerified', 'marketUrl', 'effectiveLiquidity', 'lastUpdated', 'marketPair', 'price']]
df = df[(df['isVerified'] == 1) & (df['volumePercent'] > 0.01) & (df['rank'] < 10000)]
df


4500


,rank,exchangeId,exchangeName,exchangeSlug,volumePercent,volumeQuote,isVerified,marketUrl,effectiveLiquidity,lastUpdated,marketPair,price
0,1,270,Binance,binance,12.967425,4.249748e+09,1,https://www.binance.com/en/trade/BTC_FDUSD,968.000000,2024-02-21T09:26:29.000Z,BTC/FDUSD,51453.582756
1,2,270,Binance,binance,7.680152,2.523623e+09,1,https://www.binance.com/en/trade/BTC_USDT,1057.000000,2024-02-21T09:26:22.000Z,BTC/USDT,51459.512823
2,3,294,OKX,okx,2.560399,8.413223e+08,1,https://www.okx.com/trade-spot/btc-usdt,747.000000,2024-02-21T09:26:22.000Z,BTC/USDT,51466.041763
3,4,521,Bybit,bybit,2.456321,8.069382e+08,1,https://www.bybit.com/en-US/trade/spot/BTC/USDT,721.000000,2024-02-21T09:26:29.000Z,BTC/USDT,51472.040789
4,5,89,Coinbase Exchange,coinbase-exchange,2.302601,7.564129e+08,1,https://exchange.coinbase.com/trade/BTC-USD,895.000000,2024-02-21T09:26:22.000Z,BTC/USD,51474.110000
...,...,...,...,...,...,...,...,...,...,...,...,...
3429,3430,403,BitForex,bitforex,0.017275,1.104100e+02,1,https://www.bitforex.com/en/spot/bch_btc,1.000000,2024-02-21T09:26:22.000Z,BCH/BTC,51457.789532
3434,3435,403,BitForex,bitforex,0.011849,7.573000e+01,1,https://www.bitforex.com/en/spot/dash_btc,1.000000,2024-02-21T09:26:22.000Z,DASH/BTC,51457.789537
3563,3564,908,VALR,valr,0.025038,1.555247e+08,1,https://www.valr.com/exchange/BTC/ZAR,NaN,2024-02-21T09:26:22.000Z,BTC/ZAR,52334.354873
3571,3572,1327,THORChain,thorchain-erc20,0.150284,9.712351e+06,1,https://thorchain.org/,NaN,2024-02-21T09:26:29.000Z,BTC/RUNE,51393.228865


In [18]:
grouped = df.groupby('marketPair')

min_indices = grouped['price'].idxmin()
max_indices = grouped['price'].idxmax()

min_exchange_names = df.loc[min_indices, 'exchangeName'].values
max_exchange_names = df.loc[max_indices, 'exchangeName'].values

result_df = pd.DataFrame({
    'marketPair': grouped['marketPair'].first(),
    'minPrice': df.loc[min_indices, 'price'].values,
    'minExchangeName': min_exchange_names,
    'maxPrice': df.loc[max_indices, 'price'].values,
    'maxExchangeName': max_exchange_names
})
result_df = result_df.drop_duplicates().reset_index(drop=True)
result_df

pd.w

,marketPair,minPrice,minExchangeName,maxPrice,maxExchangeName
0,ADA/BTC,51457.789382,Binance,51457.789382,Binance
1,ALT/BTC,51476.711089,Binance,51476.711089,Binance
2,AVAX/BTC,51457.789536,Binance,51457.789536,Binance
3,BCH/BTC,51457.789532,BitForex,51457.813657,IndoEx
4,BNB/BTC,51457.789530,Binance,51457.789531,Bitrue
...,...,...,...,...,...
56,XBT/USDT,51462.842282,Kraken,51462.842282,Kraken
57,XBT/ZAR,52369.367604,Luno,52369.367604,Luno
58,XMR/BTC,52180.562855,Binance,52180.562855,Binance
59,XRP/BTC,51457.789424,Cat.Ex,51457.789821,BitForex


In [23]:
result_df.to_csv('./data/bitcoin.csv')